# roadExposureCategoriesByYear #
**Author:** Andrew Larkin <br>
Created for the HEI Transit Study <br>
**Summary:** given 2 road network shapefiles with annual road and truck traffic levels, derive annual road network subsets based on traffic cutofffs

## part 1: import libraries and define global constants ##

In [1]:
import arcpy
import os
import gConst as const

In [6]:
YEARS = list(range(2007,2016))
ROAD_FOLDER = const.WIND_FOLDER + "roads/"
aadtLevels = [25000,50000,1000000000] # cutoffs for general traffic
taadtLevels = [5000,10000,1000000000] # cutoffs for truck traffic

### remove unwanted attribute fields from the shapefile attribute table ###
INPUTS: <br>
**shapefilepath** (str) - absolute filepath to the shapefile

In [52]:
def trimFields(shapeFilepath):
    fieldNames = [f.name for f in arcpy.ListFields(shapeFilepath)]
    dropFields = []
    for name in fieldNames:
        if name != "FID" and name != "Shape":
            #dropFields.append(name)
            try:
                arcpy.DeleteField_management(shapeFilepath,[name])
            except exception as e:
                print(str(e))

### for each year and traffic category, create subset files ###
INPUTS: <br>
**year** (int) - year of traffic levels to create subsets for


In [61]:
def createAnnualFiles(year):
    masterList = []
    
    # master road network file is different for 2007-2009 and 2010-2016
    masterRoadFile = ROAD_FOLDER + "rhino_vmt_2010_2016.shp" if year >= 2010 else ROAD_FOLDER + "rhino_vmt_1995_2009.shp"
    ROAD_ANNUAL_FOLDER = ROAD_FOLDER + str(year)
    if not os.path.exists(ROAD_ANNUAL_FOLDER):
        os.mkdir(ROAD_ANNUAL_FOLDER)
    
    # for each general traffic level cutoff, create a subset shapefile of roads that satisfy the range
    for cutoffIndex in range(len(aadtLevels)-1):
        outputShapefile = ROAD_ANNUAL_FOLDER + "/aadt" + str(cutoffIndex) + ".shp"
        masterList.append(outputShapefile)
        if not os.path.exists(outputShapefile):
            lowerCutoff = aadtLevels[cutoffIndex]
            upperCutoff = aadtLevels[cutoffIndex+1]
            var = "aadt" + str(year) 
            expression = var + " > " + str(lowerCutoff) 
            arcpy.Select_analysis(masterRoadFile,outputShapefile,expression)
        trimFields(outputShapefile)
        
    # for each traffic level cutoff, create a subset shapefile or roads that satisfy the range
    for cutoffIndex in range(len(taadtLevels)-1):
        outputShapefile = ROAD_ANNUAL_FOLDER + "/taadt" + str(cutoffIndex) + ".shp"
        masterList.append(outputShapefile)
        if not os.path.exists(outputShapefile):
            lowerCutoff = aadtLevels[cutoffIndex]
            upperCutoff = aadtLevels[cutoffIndex+1]
            var = "aadt_t" + str(year) 
            expression = var + " > " + str(lowerCutoff) 
            arcpy.Select_analysis(masterRoadFile,outputShapefile,expression)
        trimFields(outputShapefile)
    mergedShapefile = ROAD_ANNUAL_FOLDER + "/merged" + str(year) + ".shp"
    
    # merge all roads that satisfy at leat one cutoff range.  Used to restrict births near 500m of one of these roads
    if not os.path.exists(mergedShapefile):
        arcpy.Merge_management(masterList,mergedShapefile)

In [63]:
for year in range(2007,2016):
    createAnnualFiles(year)